In [ ]:
# --- Definir rutas ---
shapefile_path = "../data/Barrios/BARRIOS.shp"
locales_restauracion_csv = "../data/locales_restauracion.csv"

# --- Función auxiliar para asegurar que los CRS coinciden ---
def ensure_same_crs(gdf1, gdf2):
    if gdf1.crs != gdf2.crs:
        print(f"⚠️ CRS diferente detectado: {gdf1.crs} ≠ {gdf2.crs}")
        print("Reproyectando gdf2 al CRS de gdf1...")
        return gdf2.to_crs(gdf1.crs)
    return gdf2

# 1. Cargar el shapefile de barrios
gdf_barrios = gpd.read_file(shapefile_path)

# 2. Cargar locales desde el CSV de restauración
df = pd.read_csv(locales_restauracion_csv)
df = df.dropna(subset=['latitud', 'longitud'])

# 3. Convertir df a GeoDataFrame
gdf_negocios = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitud, df.latitud), crs='EPSG:4326')

# 4. Reproyectar barrios si es necesario
gdf_barrios = ensure_same_crs(gdf_negocios, gdf_barrios)

# 5. Unión espacial: asignar barrio a cada negocio
gdf_completo = gpd.sjoin(gdf_negocios, gdf_barrios, how='left', predicate='within')

# 6. Contar negocios por barrio
conteo = gdf_completo.groupby('NOMBRE').size().reset_index(name='num_negocios')
gdf_barrios = gdf_barrios.merge(conteo, on='NOMBRE', how='left')
gdf_barrios['num_negocios'] = gdf_barrios['num_negocios'].fillna(0)

# 7. Normalizar densidad
gdf_barrios['densidad_norm'] = (
    gdf_barrios['num_negocios'] - gdf_barrios['num_negocios'].min()
) / (
    gdf_barrios['num_negocios'].max() - gdf_barrios['num_negocios'].min()
)

# 8. Crear el mapa base
mapa_barrios = folium.Map(
    location=[df['latitud'].mean(), df['longitud'].mean()],
    zoom_start=12,
    tiles='https://{s}.basemaps.cartocdn.com/rastertiles/voyager_labels_under/{z}/{x}/{y}{r}.png',
    attr='Map tiles by Carto, under CC BY 3.0. Data by OpenStreetMap, under ODbL.'
)

# 9. Añadir MiniMap
MiniMap().add_to(mapa_barrios)

# 10. Crear leyenda
colormap = cm.LinearColormap(['lightblue', 'orange', 'red'], vmin=0, vmax=1)
colormap.caption = 'Densidad normalizada de locales de restauración por barrio'
colormap.add_to(mapa_barrios)

# 11. Dibujar polígonos con estilo
folium.GeoJson(
    gdf_barrios,
    name="Barrios por densidad de restauración",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['densidad_norm']),
        'color': 'black',
        'weight': 0.7,
        'fillOpacity': 0.6,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["NOMBRE", "num_negocios"],
        aliases=["Barrio", "Nº locales de restauración"],
        localize=True
    )
).add_to(mapa_barrios)

# 12. Guardar el mapa
mapa_barrios.save("../outputs/mapa_barrios_restauracion.html")
print("✅ Mapa guardado en: ../outputs/mapa_barrios_restauracion.html")

# 13. Mostrar
mapa_barrios